In [10]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

# Путь к данным
RAW_PATH = "../data/raw/"
os.makedirs("../data/processed", exist_ok=True)

print("Библиотеки загружены.")

Библиотеки загружены.


In [14]:
def excel_to_date(excel_date):
    """Конвертирует Excel-дату (44562) → datetime"""
    if pd.isna(excel_date):
        return pd.NaT
    if isinstance(excel_date, (pd.Timestamp, datetime)):
        return pd.Timestamp(excel_date)
    try:
        return datetime(1899, 12, 30) + timedelta(days=int(excel_date))
    except:
        return pd.NaT

In [15]:
files = {
    "gas": "gas.xlsx",
    "boiler_2022": "нагрузка_котельной_2022.xlsx",
    "boiler_2023_2024": "нагрузка_котельной_2023_2024.xlsx",
    "own_needs": "boiler_own_needs_2022.xlsx",
    "network_losses": "network_losses.xlsx",
    "gas_passport": "pasport_gas_2022_2024.xlsx",
    "temp_daily": "средние_температуры_по_дням.csv",
    "temp_hourly": "moscow_temp_2022_2024.csv",
    "thermal": "thermal_network_data.csv"
}

dfs = {}
for name, file in files.items():
    path = os.path.join(RAW_PATH, file)
    if not os.path.exists(path):
        print(f"Warning: {file} НЕ НАЙДЕН")
        continue
    
    print(f"Загружаем: {file}")
    if file.endswith(".xlsx"):
        df = pd.read_excel(path, sheet_name=0)
        # Находим колонку с датой
        date_cols = [col for col in df.columns if 'date' in str(col).lower()]
        if date_cols:
            date_col = date_cols[0]
            df[date_col] = df[date_col].apply(excel_to_date)
            df = df.rename(columns={date_col: 'date'})
        dfs[name] = df
    else:
        df = pd.read_csv(path)
        # CSV: timestamp или datetime
        if 'timestamp' in df.columns:
            df['date'] = pd.to_datetime(df['timestamp'])
        elif 'datetime' in df.columns:
            df['date'] = pd.to_datetime(df['datetime'])
        dfs[name] = df

print(f"\nЗагружено {len(dfs)} файлов.")

Загружаем: gas.xlsx
Загружаем: нагрузка_котельной_2022.xlsx
Загружаем: нагрузка_котельной_2023_2024.xlsx
Загружаем: boiler_own_needs_2022.xlsx
Загружаем: network_losses.xlsx
Загружаем: pasport_gas_2022_2024.xlsx
Загружаем: средние_температуры_по_дням.csv
Загружаем: moscow_temp_2022_2024.csv
Загружаем: thermal_network_data.csv

Загружено 9 файлов.


In [16]:
# Базовый индекс
all_dates = pd.date_range(start='2022-01-01', end='2024-12-31', freq='D')
daily_df = pd.DataFrame(index=all_dates).rename_axis('date')

In [17]:
if 'gas' in dfs:
    gas_df = dfs['gas'].copy()
    gas_df['date'] = gas_df['date'].dt.floor('D')
    gas_series = gas_df.set_index('date')['Vgas, m3']
    gas_series.name = 'Vgas_m3'
    daily_df = daily_df.join(gas_series, how='left')
    print("Vgas_m3 добавлен")
else:
    print("gas.xlsx не загружен")

Vgas_m3 добавлен


In [18]:
boiler_prod = pd.Series(dtype=float)

# 2022 — ежедневно
if 'boiler_2022' in dfs:
    b22 = dfs['boiler_2022'].copy()
    b22['date'] = b22['date'].dt.floor('D')
    b22_series = b22.set_index('date')['Qboiler_prod, Gcal']
    b22_series.name = 'Qboiler_prod_Gcal'
    boiler_prod = pd.concat([boiler_prod, b22_series])

# 2023-2024 — ежемесячно → ffill
if 'boiler_2023_2024' in dfs:
    b34 = dfs['boiler_2023_2024'].copy()
    b34['date'] = b34['date'].dt.to_period('M').dt.to_timestamp()
    b34_series = b34.set_index('date')['Qboiler_prod, Gcal']
    b34_series.name = 'Qboiler_prod_Gcal'
    b34_daily = b34_series.resample('D').ffill()
    boiler_prod = pd.concat([boiler_prod, b34_daily])

boiler_prod = boiler_prod[~boiler_prod.index.duplicated(keep='last')]
daily_df = daily_df.join(boiler_prod.rename('Qboiler_prod_Gcal'), how='left')
print("Qboiler_prod_Gcal добавлен")

Qboiler_prod_Gcal добавлен


C:\Temp\ipykernel_10540\2199547726.py:9: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  boiler_prod = pd.concat([boiler_prod, b22_series])


In [19]:
if 'temp_daily' in dfs:
    temp_d = dfs['temp_daily'].copy()
    temp_d['date'] = pd.to_datetime(temp_d['timestamp'])
    temp_series = temp_d.set_index('date')['air_temp']  # <-- именно air_temp
    temp_series.name = 'air_temp_daily'
    daily_df = daily_df.join(temp_series, how='left')
    print("air_temp_daily добавлен")

air_temp_daily добавлен


In [20]:
if 'gas_passport' in dfs:
    gp = dfs['gas_passport'].copy()
    gp['date'] = gp['date'].dt.to_period('M').dt.to_timestamp()
    gp_series = gp.set_index('date')['Qgas, ccal/m3']
    gp_series.name = 'Qgas_ccal_m3'
    gp_daily = gp_series.resample('D').ffill()
    daily_df = daily_df.join(gp_daily, how='left')
    print("Qgas_ccal_m3 добавлен")

Qgas_ccal_m3 добавлен


In [21]:
if 'thermal' in dfs:
    therm = dfs['thermal'].copy()
    therm['date'] = pd.to_datetime(therm['datetime']).dt.floor('D')
    t1_mean = therm.groupby('date')['T1_c'].mean()
    t2_mean = therm.groupby('date')['T2_c'].mean()
    t1_mean.name = 'T1_c_mean'
    t2_mean.name = 'T2_c_mean'
    daily_df = daily_df.join(t1_mean, how='left')
    daily_df = daily_df.join(t2_mean, how='left')
    print("T1_c_mean, T2_c_mean добавлены")

T1_c_mean, T2_c_mean добавлены


In [22]:
# Заполняем пропуски
daily_df = daily_df.ffill().bfill()

# Сохранение
output_path = "../data/processed/daily_merged.csv"
daily_df.to_csv(output_path)
print(f"\nГотово! Сохранено: {output_path}")
print(f"Строк: {len(daily_df)}, колонок: {len(daily_df.columns)}")
print("Колонки:", daily_df.columns.tolist())

# Показать первые строки
daily_df.head(10)


Готово! Сохранено: ../data/processed/daily_merged.csv
Строк: 1096, колонок: 6
Колонки: ['Vgas_m3', 'Qboiler_prod_Gcal', 'air_temp_daily', 'Qgas_ccal_m3', 'T1_c_mean', 'T2_c_mean']


,Vgas_m3,Qboiler_prod_Gcal,air_temp_daily,Qgas_ccal_m3,T1_c_mean,T2_c_mean
date,,,,,,
2022-01-01,13143.0,83.245,-1.895833,8121,91.272901,50.827322
2022-01-02,14814.0,103.855,-7.458333,8121,90.949248,50.598564
2022-01-03,14523.0,106.003,-9.795833,8121,90.042524,50.137938
2022-01-04,13295.0,101.535,-9.225000,8121,89.821026,50.010017
2022-01-05,12394.0,94.071,-7.954167,8121,89.107882,49.661876
2022-01-06,11657.0,79.148,-1.033333,8121,88.743491,49.470956
2022-01-07,15147.0,98.246,-4.779167,8121,87.862393,49.017746
2022-01-08,13025.0,104.087,-7.029167,8121,87.571059,48.872304
2022-01-09,14375.0,93.660,-5.500000,8121,86.718305,48.521103
